In [180]:
import pandas as pd
import numpy as np 

In [181]:
df = pd.read_csv('train_LSTM.csv')

In [182]:
df.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [183]:
df.__len__()

20800

In [184]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [186]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [187]:
df.dropna(inplace=True)

In [188]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [189]:
# dependent / independent 

X = df.drop('label',axis= 1)

In [190]:
y = df['label']

In [191]:
X.drop('id',axis=1,inplace=True)

In [192]:
print(X.shape,y.shape)

(18285, 3) (18285,)


## preprocessing

In [193]:
import re 
import nltk
from nltk import sent_tokenize,word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [194]:
stemmer = PorterStemmer()
lematizer = WordNetLemmatizer()

In [195]:
messages= X.copy()

In [196]:
messages.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [197]:
# len(messages['tile'])

In [198]:
corpus = []
for i in range(len(messages)):
  title = messages.iloc[i]['title']
  sent = re.sub('[^a-zA-Z]', ' ',title)
  sent = sent.lower()
  sent = sent.split()
  sent = [stemmer.stem(word) for word in sent if not word in stopwords.words('english')]
  sent = ' '.join(sent)
  corpus.append(sent)

In [199]:
len(corpus)

18285

In [200]:
import gensim
model = gensim.models.Word2Vec(window=2,min_count=1)


In [201]:
model.build_vocab(corpus)
model.train(corpus , total_examples=model.corpus_count,epochs=20)

(3314351, 19265700)

In [202]:
# model.wv(corpus)
encoded = model.wv.get_normed_vectors()

In [203]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 100
embedded_doc = pad_sequences(encoded, maxlen = sent_length, padding = 'pre')

In [204]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [205]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
voc_size = 5000   
onehot_repe = [one_hot(words,voc_size)for words in corpus]

sent_length = 50
embedded_doc = pad_sequences(onehot_repe, maxlen = sent_length, padding = 'pre')


In [206]:
len(embedded_doc[0])

50

In [207]:
len(embedded_doc)

18285

## LSTM

In [208]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Assuming voc_size is defined elsewhere
embedding_vector_features = 50
voc_size = 5000

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features))
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary
print(model.summary())


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_23 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [209]:
import numpy as np
X_input = np.array(embedded_doc)
y_input = np.array(y)

In [210]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_input,y_input, test_size = 0.33, random_state = 42 )

In [211]:

model.fit(X_train,y_train, epochs = 5, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/5


192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step - accuracy: 0.7860 - loss: 0.3986 - val_accuracy: 0.9157 - val_loss: 0.1961
Epoch 2/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step - accuracy: 0.9501 - loss: 0.1239 - val_accuracy: 0.9200 - val_loss: 0.1902
Epoch 3/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step - accuracy: 0.9705 - loss: 0.0844 - val_accuracy: 0.9170 - val_loss: 0.2161
Epoch 4/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 93ms/step - accuracy: 0.9819 - loss: 0.0584 - val_accuracy: 0.9193 - val_loss: 0.2497
Epoch 5/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 97ms/step - accuracy: 0.9853 - loss: 0.0415 - val_accuracy: 0.9160 - val_loss: 0.3336


In [212]:

y_pred = (model.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


(6035, 1)

In [213]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3065,  354],
       [ 153, 2463]])

In [214]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.915990057995029

## Bidirectional LSTM

In [215]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional,Dense,LSTM

In [216]:
model1 = Sequential()
model1.add(Embedding(input_dim=5000,output_dim=50))
model1.add(Bidirectional(LSTM(100)))
# model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='softmax'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
model1.summary()


Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [217]:
model1.fit(X_train,y_train,epochs=1,batch_size=128,validation_data=(X_test,y_test))

96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - acc: 0.4307 - loss: 0.5007 - val_acc: 0.4335 - val_loss: 0.2060


In [218]:
y_pred = (model1.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


(6035, 1)

In [219]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   0, 3419],
       [   0, 2616]])

In [220]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.4334714167357084

## RNN

In [221]:
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN

model2 = Sequential()
model2.add(Embedding(input_dim=5000,output_dim=50))
model2.add(SimpleRNN(100))
model2.add(Dense(1,activation = 'sigmoid'))

model2.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model2.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_25 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [222]:
model2.fit(X_train,y_train, epochs = 2, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/2


192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8415 - loss: 0.3540 - val_accuracy: 0.9181 - val_loss: 0.1847
Epoch 2/2
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9572 - loss: 0.1149 - val_accuracy: 0.9198 - val_loss: 0.1881


In [223]:
# Prediction
y_pred = (model2.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


(6035, 1)

In [224]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[3152,  267],
       [ 217, 2399]])

In [225]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.91980115990058

## Bidirectional RNN

In [226]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, SimpleRNN,Dropout

model3 = Sequential()
model3.add(Embedding(input_dim=5000,output_dim=50))
model3.add(Bidirectional(SimpleRNN(100)))
model3.add(Dense(1,activation = 'sigmoid'))
model3.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model3.summary()

Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_26 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [227]:
model3.fit(X_test,y_test,epochs=2,validation_data=(X_test,y_test))

Epoch 1/2


189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8383 - loss: 0.3673 - val_accuracy: 0.9084 - val_loss: 0.2117
Epoch 2/2
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9203 - loss: 0.1908 - val_accuracy: 0.9162 - val_loss: 0.2047


In [228]:
y_pred = (model2.predict(X_test)>0.5).astype("int64")

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [229]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [230]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [231]:
confusion_matrix(y_test, y_pred)


array([[3152,  267],
       [ 217, 2399]])

In [232]:
accuracy_score(y_test, y_pred)

0.91980115990058

In [233]:
model4 = Sequential()
model4.add(Embedding(input_dim=5000,output_dim=50))
model4.add(Dropout(0.3))
model4.add(Bidirectional(LSTM(100)))

model4.add(Dense(1,activation='softmax'))
model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
model4.summary()


Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_27 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [234]:
model4.fit(X_test,y_test,epochs=2,validation_data=(X_test,y_test))

Epoch 1/2


189/189 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - acc: 0.4414 - loss: 0.4698 - val_acc: 0.4335 - val_loss: 0.1618
Epoch 2/2
189/189 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - acc: 0.4289 - loss: 0.1609 - val_acc: 0.4335 - val_loss: 0.0939


In [235]:
# Prediction
y_pred = (model4.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


(6035, 1)

In [236]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   0, 3419],
       [   0, 2616]])

In [237]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.4334714167357084